In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML

In [2]:
dir_ = '../../output/stage_2_predictions/Keck_Pria_AS_Retest'
complete_df = pd.read_csv('{}/complete.csv'.format(dir_))

## Show 84 Active Compounds' Corresponding Predicted Values

In [3]:
dataframe = pd.read_excel('../../output/stage_2_predictions/Keck_LC4_export.xlsx')

supplier_id = dataframe['Supplier ID'].tolist()
failed_id = ['F0401-0050', 'F2964-1411', 'F2964-1523']
inhibits = dataframe[
    'PriA-SSB AS, normalized for plate and edge effects, correct plate map: % inhibition Alpha, normalized (%)'].tolist()

positive_enumerate = filter(lambda x: x[1] >= 35 and supplier_id[x[0]] not in failed_id, enumerate(inhibits))
positive_idx = map(lambda x: x[0], positive_enumerate)

In [4]:
model_names = complete_df.columns[3:]

In [5]:
columns = ['inhibition']
columns.extend(model_names)
HTML(complete_df[complete_df['label']>0][columns].to_html())

,inhibition,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,LightChem_a,LightChem_b,LightChem_c,LightChem_d,LightChem_e,LightChem_f,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
456,93.622204,0.032070,9.558674,8.300441,7.915640,0.035326,4.65,29.266851,7.039242,-0.235210,58.0847,14.804,35.913,5.871,3.82,0.500282,0.499869,0.499749,0.501856,0.507566,0.002003,0.041430,0.794751,0.502844,0.630001,0.633506,0.316261,0.451398,1.306898e-01,1.417417e-02,0.63800,0.627500,0.633875,0.63925,0.629500,0.66100,0.652000,0.999364,0.745777,111.501968,115.448441
483,83.123806,0.026200,11.627432,10.401617,10.348252,0.052207,6.83,49.549496,10.348237,-0.216938,96.0663,16.759,38.103,8.955,7.17,0.500282,0.499869,0.504866,0.505282,0.511596,0.002003,0.041506,0.041104,0.013553,0.024641,0.123456,0.012172,0.020675,8.293058e-05,5.710558e-05,0.26250,0.263750,0.261125,0.20825,0.209625,0.16450,0.160625,0.000011,0.013834,-3.408564,0.307884
1239,72.063678,0.020902,10.643016,9.375816,9.086127,0.026928,6.72,41.775131,8.748974,0.820223,81.6115,14.474,34.089,8.930,3.36,0.500282,0.522985,0.545548,0.543723,0.554211,0.002003,0.041435,0.758333,0.274311,0.429875,0.633105,0.192712,0.250155,2.490977e-01,2.412796e-01,0.04125,0.039250,0.040313,0.04925,0.054250,0.11950,0.118250,0.165573,0.206531,29.097578,85.389671
1878,89.493228,0.027703,12.084562,10.107620,10.038560,0.039552,6.65,40.496170,11.273830,2.226941,79.9549,16.883,37.892,8.525,4.85,0.710454,0.575938,0.595639,0.592230,0.601834,0.001995,0.041460,0.629179,0.650062,0.581717,0.739583,0.445657,0.551192,2.596361e-04,4.096619e-03,0.62775,0.629375,0.631313,0.55125,0.555625,0.61225,0.606500,0.000062,0.025602,4.825106,13.609948
2179,44.850520,0.136960,13.457431,10.838761,10.761808,0.154201,7.66,43.351162,9.481766,-0.060435,89.0793,27.440,47.976,7.494,8.44,0.500282,0.499869,0.499749,0.500670,0.505504,0.001990,0.041254,0.009812,0.011348,0.030020,0.016756,0.007038,0.011007,8.081179e-09,4.435486e-07,0.23500,0.231750,0.230875,0.23600,0.239375,0.23175,0.233625,0.000035,0.012207,-5.790788,-4.602567
2309,93.499335,0.100197,12.998991,11.054573,10.995590,0.161404,6.55,48.574375,12.967777,1.797404,88.8323,20.722,41.832,9.279,6.81,0.500282,0.511202,0.533921,0.532971,0.542782,0.002001,0.041641,0.053362,0.054569,0.088330,0.068429,0.051830,0.066466,3.407333e-05,1.647835e-04,0.01600,0.017875,0.016125,0.01975,0.021000,0.05475,0.052625,0.000329,0.109288,6.454531,12.292480
3179,71.993342,0.014772,11.898092,10.060977,9.957698,0.025295,6.20,42.910912,10.002242,0.058531,74.8190,21.441,44.496,7.839,6.04,0.696237,0.562904,0.574395,0.574022,0.581741,0.002005,0.041476,0.365992,0.114606,0.340758,0.319927,0.083147,0.129246,6.132637e-02,3.185447e-02,0.09375,0.098250,0.101437,0.16750,0.161875,0.33125,0.325750,0.042266,0.161456,15.114231,46.100803
3327,69.353268,0.272675,13.327268,11.368801,11.679024,0.300917,9.49,61.445744,8.386213,-2.044100,92.4668,22.579,47.814,10.382,8.16,0.500282,0.499869,0.499749,0.500670,0.505504,0.002000,0.041377,0.000907,0.000678,0.000724,0.000644,0.001223,0.001174,6.992787e-08,2.628636e-07,0.00050,0.000375,0.000500,0.00100,0.000750,0.00150,0.001000,0.000004,0.016198,-1.251667,-3.816286
3335,55.856874,0.009858,10.433755,9.889037,9.891108,0.017578,6.27,43.284203,10.395682,-0.494275,85.9378,17.885,38.858,8.215,5.64,0.500282,0.499869,0.499749,0.500670,0.505504,0.002002,0.041470,0.000950,0.000751,0.000791,0.000710,0.001175,0.001227,2.231181e-05,6.569731e-06,0.00300,0.002000,0.001687,0.00225,0.002000,0.00125,0.000875,0.000021,0.023615,-2.002434,-3.494677
3419,67.383500,0.100206,12.065390,10.921904,11.003480,0

## Show 84 Active Compounds' Corresponding Predicted Rank

In [6]:
rank_df = complete_df[['molecule id', 'label', 'inhibition']]

for (idx, model_name) in enumerate(model_names):
    order = complete_df[model_name].rank().tolist()
    order = np.array(order)
    order = order.astype(np.int)
    
    rank_df[model_name]=order
    

/usr/local/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
HTML(rank_df[rank_df['label']>0].to_html())

,molecule id,label,inhibition,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,LightChem_a,LightChem_b,LightChem_c,LightChem_d,LightChem_e,LightChem_f,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
456,F0447-0233,1,93.622204,14309,694,587,349,10243,839,378,679,7506,448,3337,5333,330,686,12589,12545,12464,24798,24801,21712,18203,25278,25277,25278,25277,25277,25277,25274,25267,25277,25275,25276,25277,25276,25277,25277,25277,25277,25278,25279
483,F0472-0070,1,83.123806,11819,17067,17219,15986,15247,13141,17743,15148,7663,23851,8600,11936,18301,19370,12589,12545,24954,24946,24944,23082,23400,25242,25215,25219,25261,25198,25224,23570,23536,25270,25270,25270,25267,25267,25257,25256,1082,13570,15764,20739
1239,F0696-4000,1,72.063678,8681,4750,3398,2382,6551,12016,4404,3635,18187,11156,2761,2102,18065,278,12589,25165,25202,25189,25201,22184,18720,25277,25275,25275,25276,25276,25276,25276,25277,25200,25200,25200,25204,25213,25247,25247,25267,25263,25219,25274
1878,F1031-0163,1,89.493228,12554,21525,11978,9941,11737,11277,3238,22553,24712,9476,8996,11275,13558,3261,25206,25267,25265,25264,25264,6423,20944,25276,25278,25277,25278,25278,25279,24659,25242,25275,25276,25275,25275,25275,25275,25275,20821,23246,24714,24908
2179,F1243-0026,1,44.850520,23922,25073,23076,21950,23421,20533,6372,7512,9103,19273,25255,25112,3905,24431,12589,12545,12464,12299,11932,1889,3017,25200,25205,25226,25225,25135,25176,785,6627,25269,25268,25268,25270,25270,25267,25267,12716,10921,7255,12272
2309,F1310-0123,1,93.499335,22883,24626,24385,23751,23580,10297,15958,25182,23874,19032,21311,21982,21039,16767,12589,25106,25164,25163,25167,15099,24983,25247,25254,25249,25251,25259,25259,21809,24683,25121,25132,25123,25143,25149,25211,25209,25121,25237,24843,24845
3179,F1835-0132,1,71.993342,3475,19894,11164,8792,5644,7125,5779,11031,10483,4929,22628,24435,6297,10443,25200,25260,25254,25254,25253,25053,22130,25270,25264,25272,25271,25267,25270,25267,25272,25238,25243,25244,25263,25261,25272,25272,25262,25253,25128,25244
3327,F1883-1660,1,69.353268,25014,24986,25074,25205,24940,24137,25053,2519,670,22072,24000,25099,24969,23885,12589,12545,12464,12299,11932,11106,11211,17907,14875,17025,15200,18437,17795,2550,5172,19534,18451,19826,21157,20010,23171,22140,45,16915,21644,13878
3335,F1885-0026,1,55.856874,82,3158,8547,8176,1213,7725,6264,15671,5521,16129,12631,14492,10033,7455,12589,12545,12464,12299,11932,18072,21675,19037,18074,19545,18185,17087,19147,20685,17201,24319,23773,23518,23695,23567,22684,21627,5438,22538,20119,14500
3419,F1899-1464,1,67.383500,22885,21357,23716,23799,23679,9923,20978,21477,14681,22486,21066,23124,13963,24677,12589,12545,12464,12299,24380,21618,11305,17441,21083,18986,17163,18092,17298,22603,18738,16335,13289,10287,13297,9618,18594,17235,3761,14345,13604,21089


## Show n-hits in top 250

In [8]:
model_names = complete_df.columns[3:]

In [9]:
pos_df = complete_df[complete_df['label']> 0]
inhibitions = pos_df['inhibition'].tolist()

N = len(pos_df)

predicted_values = [[] for _ in model_names]
for (idx, selected_model) in enumerate(model_names):
    predicted_values[idx] = pos_df[selected_model].tolist()
    all_predicted = complete_df[selected_model].tolist()
    all_predicted = np.array(all_predicted)
    index = all_predicted.argsort()[::-1]
    top_index = index[:250]
    hits = np.intersect1d(positive_idx, top_index)
    print selected_model ,':\t', len(hits)

ConsensusDocking_efr1_opt :	0
ConsensusDocking_max :	2
ConsensusDocking_mean :	1
ConsensusDocking_median :	2
ConsensusDocking_rocauc_opt :	0
Docking_ad4 :	2
Docking_dock6 :	3
Docking_fred :	2
Docking_hybrid :	2
Docking_plants :	1
Docking_rdockint :	2
Docking_rdocktot :	2
Docking_smina :	0
Docking_surflex :	1
IRV_a :	17
IRV_b :	25
IRV_c :	30
IRV_d :	30
IRV_e :	30
LSTM_a :	1
LSTM_b :	1
LightChem_a :	47
LightChem_b :	50
LightChem_c :	48
LightChem_d :	45
LightChem_e :	47
LightChem_f :	48
MultiClassification_a :	26
MultiClassification_b :	31
RandomForest_a :	39
RandomForest_b :	39
RandomForest_c :	39
RandomForest_d :	38
RandomForest_e :	39
RandomForest_f :	40
RandomForest_g :	41
SingleClassification_a :	25
SingleClassification_b :	34
SingleRegression_a :	35
SingleRegression_b :	35
